In [1]:
import requests
from bs4 import BeautifulSoup

url = "https://www.comebuy2002.com.tw/store/"
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")
articles = soup.find_all("li", class_="g-mv-12 clearfix")

for a in articles:
    store_name_tag = a.find("div", class_="itemBox storeTitle")
    store_name = store_name_tag.text.strip() if store_name_tag else "沒有店名"
    
    store_address_tag = a.find("div", class_="itemBox address")
    store_address = store_address_tag.text.strip() if store_address_tag else "沒有地址"
    
    store_phone_tag = a.find("div", class_="itemBox tel")
    store_phone = store_phone_tag.find("a").text.strip() if store_phone_tag and store_phone_tag.find("a") else "沒有電話"
    
    print(f"店名: {store_name}")
    print(f"地址: {store_address}")
    print(f"電話: {store_phone}")
    print("-" * 30)


    

店名: 三重台北橋(籌備中)
地址: 新北市三重區文化南路6-1號
電話: 沒有電話
------------------------------
店名: 中壢莊敬(籌備中)
地址: 桃園市中壢區莊敬路777號
電話: 沒有電話
------------------------------
店名: 蘆洲中正(籌備中)
地址: 新北市蘆洲區中正路187號
電話: 沒有電話
------------------------------
店名: 台中昌平(籌備中)
地址: 台中市北屯區昌平路一段350號
電話: 沒有電話
------------------------------
店名: 士林承德
地址: 台北市士林區承德路四段40巷125號
電話: 02-2889-1219
------------------------------
店名: 台中太平
地址: 台中市太平區太平里太平路593號1樓
電話: 04-2276-0789
------------------------------
店名: 蘆洲民族
地址: 新北市蘆洲區民族路498號
電話: 02-2848-8781
------------------------------
店名: 樹林秀泰
地址: 新北市板橋區大觀路三段160巷34弄2號
電話: 02-2825-0807
------------------------------
店名: 板橋致理
地址: 新北市板橋區文化路一段313號(誠信館)
電話: 02-8257-0211
------------------------------
店名: 瑞芳站前
地址: 新北市瑞芳區民生街11號
電話: 02-2496-8866
------------------------------
店名: 板橋北門
地址: 新北市板橋區文化路一段17號
電話: 02-2968-6097
------------------------------
店名: 台北教育大學
地址: 台北市大安區敦南街38號
電話: 02-2736-0050
------------------------------
店名: 新莊雙鳳
地址: 新北市新莊區雙鳳路25號
電話: 02-2907-1066
------------------------------
店名: 台北中華


In [3]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import pyodbc

# 第一步：爬取數據並寫入 CSV 文件
url = "https://www.comebuy2002.com.tw/store/"
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")
articles = soup.find_all("li", class_="g-mv-12 clearfix")

stores = []
for a in articles:
    store_name_tag = a.find("div", class_="itemBox storeTitle")
    store_name = store_name_tag.text.strip() if store_name_tag else "沒有店名"
    
    store_address_tag = a.find("div", class_="itemBox address")
    store_address = store_address_tag.text.strip() if store_address_tag else "沒有地址"
   
    store_phone_tag = a.find("div", class_="itemBox tel")
    store_phone = store_phone_tag.find("a").text.strip() if store_phone_tag and store_phone_tag.find("a") else "沒有電話"
    
    stores.append([store_name, store_address, store_phone])


conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                    'Server=LAPTOP-QV5BBV9V\SQLEXPRESS;'
                    'Database=comebuy;'
                    'Trusted_Connection=yes;')

sql='''CREATE TABLE comebuy (ShopName varchar(50), FullAddress nvarchar(150), Tel nvarchar(50));'''
conn.execute(sql) #執行sql指令
conn.commit()
conn.close()

# 將數據寫入 CSV 文件
csv_file = 'c:/data/stores.csv'
with open(csv_file, mode='w', newline='', encoding='utf-8-sig') as file:
    writer = csv.writer(file)
    writer.writerow(["store_name", "store_address", "store_phone"])  # 寫入表頭
    writer.writerows(stores)

print(f"數據已成功寫入 {csv_file} 文件。")

# 第二步：將 CSV 文件中的數據寫入 SQL Server 資料庫
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=LAPTOP-QV5BBV9V\SQLEXPRESS;'
                      'Database=comebuy;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()

# 確認表已存在，如果不存在則創建
cursor.execute('''
IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='comebuy' AND xtype='U')
CREATE TABLE comebuy (
    id INT IDENTITY(1,1) PRIMARY KEY,
    ShopName NVARCHAR(255),
    FullAddress NVARCHAR(255),
    Tel NVARCHAR(50)
)
''')

# 從 CSV 文件讀取數據
df = pd.read_csv(csv_file, encoding='utf-8-sig')

# 插入數據
for index, row in df.iterrows():
    cursor.execute("INSERT INTO comebuy (ShopName, FullAddress, Tel) VALUES (?, ?, ?)", 
                   row.store_name, row.store_address, row.store_phone)

conn.commit()
cursor.close()
conn.close()

print("數據已成功存儲到 SQL Server 資料庫中。")

Error: ('01000', "[01000] [unixODBC][Driver Manager]Can't open lib 'ODBC Driver 17 for SQL Server' : file not found (0) (SQLDriverConnect)")

In [4]:
import pyodbc
import pandas as pd
import matplotlib.pyplot as plt

# 連接到 SQL Server 資料庫
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=LAPTOP-QV5BBV9V\\SQLEXPRESS;'
                      'Database=comebuy;'
                      'Trusted_Connection=yes;')

# 從資料庫中選擇數據
sql = '''SELECT * FROM comebuy'''
df = pd.read_sql(sql, conn)

# 關閉資料庫連接
conn.close()

# 去除重複的店家
df.drop_duplicates(subset=['ShopName', 'FullAddress'], inplace=True)

# 顯示前幾行數據
print(df.head())

# 定義各地區包含的縣市
regions = {
    '北部': ['台北市', '新北市', '基隆市', '桃園市', '新竹市', '新竹縣'],
    '中部': ['台中市', '苗栗縣', '彰化縣', '南投縣', '雲林縣'],
    '南部': ['台南市', '高雄市', '嘉義市', '嘉義縣', '屏東縣'],
    '東部': ['宜蘭縣', '花蓮縣', '台東縣'],
    '外島': ['澎湖縣', '金門縣', '連江縣', '綠島縣', '蘭嶼縣']
}

# 解析地址以提取縣市並進行地區分類
def extract_region(address):
    for region, cities in regions.items():
        if any(city in address for city in cities):
            return region
    return '其他'

# 提取地區信息
df['Region'] = df['FullAddress'].apply(extract_region)

# 去除“其他”分類
df = df[df['Region'] != '其他']

# 計算每個地區的店家數量
region_counts = df['Region'].value_counts()

# 設置字體以解決中文顯示問題
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei']  # 使用 Microsoft JhengHei 字體
plt.rcParams['axes.unicode_minus'] = False

# 繪製地區店家數量分布長條圖
plt.figure(figsize=(12, 8))
ax = region_counts.plot(kind='bar')
plt.title('各地區店家數量分布')
plt.xlabel('地區')
plt.ylabel('店家數量')
plt.xticks(rotation=0)

# 在長條圖上添加數量標籤
for container in ax.containers:
    ax.bar_label(container)

plt.show()

# 繪製地區店家數量分布圓餅圖
plt.figure(figsize=(10, 10))
plt.pie(region_counts, labels=region_counts.index, autopct='%1.1f%%', startangle=140)
plt.title('各地區店家數量分布')
plt.ylabel('')
plt.show()





Error: ('01000', "[01000] [unixODBC][Driver Manager]Can't open lib 'ODBC Driver 17 for SQL Server' : file not found (0) (SQLDriverConnect)")

In [5]:
import requests
from bs4 import BeautifulSoup
import csv

url = "https://www.comebuy2002.com.tw/store/"
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")
articles = soup.find_all("li", class_="g-mv-12 clearfix")

stores = []
for a in articles:
    store_name_tag = a.find("div", class_="itemBox storeTitle")
    store_name = store_name_tag.text.strip() if store_name_tag else "沒有店名"
    
    store_address_tag = a.find("div", class_="itemBox address")
    store_address = store_address_tag.text.strip() if store_address_tag else "沒有地址"
   
    store_phone_tag = a.find("div", class_="itemBox tel")
    store_phone = store_phone_tag.find("a").text.strip() if store_phone_tag and store_phone_tag.find("a") else "沒有電話"
    
    stores.append([store_name, store_address, store_phone])

# 將數據寫入 CSV 文件
csv_file = 'stores.csv'
with open(csv_file, mode='w', newline='', encoding='utf-8-sig') as file:
    writer = csv.writer(file)
    writer.writerow(["store_name", "store_address", "store_phone"])  # 寫入表頭
    writer.writerows(stores)

print(f"數據已成功寫入 {csv_file} 文件。")


數據已成功寫入 stores.csv 文件。


In [8]:
from flask import Flask, render_template
import pyodbc
import pandas as pd
import matplotlib.pyplot as plt
import io
import base64
import traceback
from werkzeug.serving import run_simple

app = Flask(__name__)

def get_data():
    try:
        conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                              'Server=LAPTOP-QV5BBV9V\\SQLEXPRESS;'
                              'Database=comebuy;'
                              'Trusted_Connection=yes;')
        sql = '''SELECT * FROM comebuy'''
        df = pd.read_sql(sql, conn)
        conn.close()
        return df
    except Exception as e:
        print(f"Error in get_data: {e}\n{traceback.format_exc()}")
        raise

def create_charts(df):
    regions = {
        '北部': ['台北市', '新北市', '基隆市', '桃園市', '新竹市', '新竹縣'],
        '中部': ['台中市', '苗栗縣', '彰化縣', '南投縣', '雲林縣'],
        '南部': ['台南市', '高雄市', '嘉義市', '嘉義縣', '屏東縣'],
        '東部': ['宜蘭縣', '花蓮縣', '台東縣'],
        '外島': ['澎湖縣', '金門縣', '連江縣']
    }

    def extract_region(address):
        for region, cities in regions.items():
            if any(city in address for city in cities):
                return region
        return '其他'

    df['Region'] = df['FullAddress'].apply(extract_region)
    df.drop_duplicates(subset=['ShopName', 'FullAddress'], inplace=True)
    region_counts = df['Region'].value_counts()

    plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei']
    plt.rcParams['axes.unicode_minus'] = False

    # 長條圖
    labels = ['北部', '中部', '南部', '東部', '外島']
    sizes = [region_counts.get('北部', 0), region_counts.get('中部', 0), region_counts.get('南部', 0), region_counts.get('東部', 0), region_counts.get('外島', 0)]
    
    plt.figure(figsize=(12, 8))
    plt.bar(labels, sizes)
    plt.title('各地區店家數量分布')
    plt.xlabel('地區')
    plt.ylabel('店家數量')
    for i, v in enumerate(sizes):
        plt.text(i, v + 0.5, str(v), ha='center', va='bottom')
    
    bar_chart = io.BytesIO()
    plt.savefig(bar_chart, format='png')
    bar_chart.seek(0)
    bar_chart_base64 = base64.b64encode(bar_chart.getvalue()).decode()

    # 圓餅圖
    plt.figure(figsize=(10, 10))
    plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140)
    plt.title('各地區店家數量比例')
    
    pie_chart = io.BytesIO()
    plt.savefig(pie_chart, format='png')
    pie_chart.seek(0)
    pie_chart_base64 = base64.b64encode(pie_chart.getvalue()).decode()

    return bar_chart_base64, pie_chart_base64

@app.route('/')
def index():
    try:
        df = get_data()
        bar_chart, pie_chart = create_charts(df)
        return render_template('index.html', tables=df.to_html(classes='data', header="true", index=False), bar_chart=bar_chart, pie_chart=pie_chart)
    except Exception as e:
        error_message = f"An error occurred: {e}\n{traceback.format_exc()}"
        print(error_message)
        return render_template('index.html', error_message=error_message)

if __name__ == '__main__':
    try:
        run_simple('0.0.0.0', 5000, app, use_reloader=False)
    except Exception as e:
        print(f"An error occurred during startup: {e}\n{traceback.format_exc()}")





 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.16.5.4:5000
Press CTRL+C to quit
127.0.0.1 - - [29/Jun/2024 08:22:41] "GET / HTTP/1.1" 200 -


Error in get_data: ('01000', "[01000] [unixODBC][Driver Manager]Can't open lib 'ODBC Driver 17 for SQL Server' : file not found (0) (SQLDriverConnect)")
Traceback (most recent call last):
  File "/tmp/ipykernel_6847/3558616833.py", line 14, in get_data
    conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
pyodbc.Error: ('01000', "[01000] [unixODBC][Driver Manager]Can't open lib 'ODBC Driver 17 for SQL Server' : file not found (0) (SQLDriverConnect)")

An error occurred: ('01000', "[01000] [unixODBC][Driver Manager]Can't open lib 'ODBC Driver 17 for SQL Server' : file not found (0) (SQLDriverConnect)")
Traceback (most recent call last):
  File "/tmp/ipykernel_6847/3558616833.py", line 80, in index
    df = get_data()
  File "/tmp/ipykernel_6847/3558616833.py", line 14, in get_data
    conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
pyodbc.Error: ('01000', "[01000] [unixODBC][Driver Manager]Can't open lib 'ODBC Driver 17 for SQL Server' : file not found (0) 

In [ ]:
import threading

def run_flask():
    run_simple('0.0.0.0', 5000, app, use_reloader=False)

thread = threading.Thread(target=run_flask)
thread.start()




 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.104:5000
Press CTRL+C to quit
